<a href="https://colab.research.google.com/github/fachiny17/machine_learning/blob/main/tensorflow_course/my_food_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This is the version of `MY FOOD CLASSIFIER`

Note: The dataset is from TensorFlow Datsets

In [4]:
# Import the necessary libraries and packages

import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive
import os

## 1. Get and load the dataset

In [5]:
# Get all datasets in tensorflow_datasets and see if our dataset is there
dataset_list = tfds.list_builders()
# Check if our dataset exists
print(f"Does our dataset exist? {"food101" in dataset_list}")

Does our dataset exist? True


In [6]:
# Mount Google Drive
drive.mount('/content/drive')

# Set download to Google Drive
data_dir = '/content/drive/MyDrive/datasets/food101'

Mounted at /content/drive


In [7]:
# # Import and download the data (first tome or initial download)
# (train_data, test_data), ds_info = tfds.load(name="food101",
#                                              split=["train", "validation"],
#                                              shuffle_files=True,
#                                              as_supervised=True,
#                                              with_info=True,
#                                              data_dir=data_dir,
#                                              download=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /content/drive/MyDrive/datasets/food101/food101/incomplete.CIAV4V_2.0.0/food101-train.tfrecord*...: …

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /content/drive/MyDrive/datasets/food101/food101/incomplete.CIAV4V_2.0.0/food101-validation.tfrecord*…

Dataset food101 downloaded and prepared to /content/drive/MyDrive/datasets/food101/food101/2.0.0. Subsequent calls will reuse this data.


In [ ]:
# Import and load the data
(train_data, test_data), ds_info = tfds.load(name="food101",
                                             split=["train", "validation"],
                                             shuffle_files=True,
                                             as_supervised=True,
                                             with_info=True,
                                             data_dir=data_dir)

In [ ]:
# Features of Food101 TFDS
ds_info.features

In [ ]:
# Get class names
class_names = ds_info.features["label"].names
class_names[:20]

In [ ]:
# Take one sample of the training data
train_one_sample = train_data.take(1)
train_one_sample

In [ ]:
# Output info about the our training sample
for image, label in train_one_sample:
  print(f"""
  Image shape: {image.shape}
  Image dtype: {image.dtype},tensor form): {label}
  Class name (str form): {class_names[label]}
  """)

In [ ]:
image

In [ ]:
# Plot an image from TensorFlow Datasets
plt.imshow(image)
plt.title(class_names[label])
#plt.axis(False);

## 2. Create preprocessing functions for our data

In [ ]:
# Make a function for preprocessing images
def preprocess_img(image, label, img_shape=224):
  """
  Converts image datatype from 'uint8' -> 'float32' and reshapes image to [img_shape, img_shape, color_channels]
  """
  image = tf.image.resize(image, [img_shape, img_shape]) # reshape to img_shape
  return tf.cast(image, tf.float32), label # return (float32_image, label)

In [ ]:
# Preprocess a single sample image and check the outputs
preprocessed_img = preprocess_img(image, label)[0]
print(f"Image before preprocessing:\n {image[:2]}...,\nShape: {image.shape},\nDatatype: {image.dtype}\n")
print(f"Image after preprocessing:\n {preprocessed_img[:2]}...\nShape: {preprocessed_img.shape},\nDatatype: {preprocessed_img.dtype}")

In [ ]:
# Plot the preprocessed image
plt.imshow(preprocessed_img/255.)
plt.title(class_names[label])
#plt.axis(False);

## 3. Batch and Prepare datasets

In [ ]:
# Map preprocessing fuction to training data (and paralellize)
